<a href="https://colab.research.google.com/github/natalievolk/LearnAI/blob/training/Copy_of_LearnAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader,TensorDataset
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
import torchvision
import torchvision.transforms as transforms

# pillow to load image
from PIL import Image
import cv2
import requests
from io import BytesIO
#import urllib2

In [67]:
# training data
url_train = 'https://raw.githubusercontent.com/natalievolk/LearnAI/main/original-datasets/book30-listing-train.csv'
data_train = pd.read_csv(url_train, index_col=False, usecols=[2,5], encoding = "ISO-8859-1")
data_train.columns = ['jpg_url', 'classification']


# testing data
url_test = 'https://raw.githubusercontent.com/natalievolk/LearnAI/main/original-datasets/book30-listing-test.csv'
data_test = pd.read_csv(url_test, index_col=False, usecols=[2,5], encoding = "ISO-8859-1")
data_test.columns = ['jpg_url', 'classification']


In [68]:
# MAKING TRAINING SET

file = open("train.txt", 'w')
#Image Preprocessing
 
for index, image_name in enumerate(data_train['jpg_url']):
    if image_name[-4:] == ".jpg":
      file.write(image_name + " " + str(data_train['classification'][index]) + "\n")

file.close()


file = open("test.txt",'w')
 
for index, image_name in enumerate(data_test['jpg_url']):
    if image_name[-4:] == ".jpg":
      file.write(image_name + " " + str(data_test['classification'][index]) + "\n")
 
file.close()


In [69]:
class ImagesDataset(Dataset):

    def __init__(self, text_file):
        self.df = pd.read_csv(text_file, sep=' ')
        self.df.columns = ['jpg_url', 'classification']

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        response = requests.get(self.df['jpg_url'][idx])
        img = Image.open(BytesIO(response.content)).resize((400, 400))
        pix = np.array(img)
 
        return (pix, self.df['classification'][idx])


In [70]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        if self.transform:
            x = Image.fromarray(self.data[index].astype(np.uint8).transpose(1,2,0))
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

In [80]:
Train_dataset = ImagesDataset("train.txt")
train_data = []
train_targets = []

for i in range(20): #len(train_dataset)
  train_data.append(Train_dataset[i][0])
  train_targets.append(Train_dataset[i][1])

dataset_train = MyDataset(data, targets)
train_dataloader = DataLoader(dataset_train, batch_size=32, num_workers=4)

for a,b in enumerate(train_dataloader):
  print(a,b)


RuntimeError: ignored

In [77]:
Test_dataset = ImagesDataset("test.txt")
test_data = []
test_targets = []

for i in range(20): ##len(train_dataset)
  test_data.append(Test_dataset[i][0])
  test_targets.append(Test_dataset[i][1])

dataset_test = MyDataset(data, targets)
test_dataloader = DataLoader(dataset_test, batch_size=32, num_workers=4)


In [73]:

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            # in_channels = 3 (RGB), out_channels = 6
            Conv2d(3, 6, kernel_size=5, stride=2, padding=0),
            BatchNorm2d(6), # parameter = out_channels
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(6, 12, kernel_size=7, stride=2, padding=0),
            BatchNorm2d(12),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout(0.2, inplace=True)
        )

        self.linear_layers = Sequential(
            Linear(12 * 48 * 48, 128),
            Linear(128, 64),
            Linear(64, 32)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x


In [74]:
# defining the model
net = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
  

In [79]:
#Training Stage
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.reshape(20, 3, 400, 400) #Change later

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: ignored